# Train Model on Distributed Cluster

# STOP ALL KERNELS BEFORE CONTINUING!!
# THE GPU GETS WEDGED AT THIS POINT!!!

## Define Cluster Spec

In [2]:
import tensorflow as tf

cluster = tf.train.ClusterSpec({"local": ["localhost:2222", "localhost:2223"]})

## Start Server "Task 0" (localhost:2222)

In [3]:
server0 = tf.train.Server(cluster, job_name="local", task_index=0)

print(server0)

## Start Server "Task 1" (localhost:2223)

In [4]:
server1 = tf.train.Server(cluster, job_name="local", task_index=1)

print(server1)

## Define a Computationally-intensive TensorFlow Graph

In [5]:
import tensorflow as tf

n = 2
c1 = tf.Variable([])
c2 = tf.Variable([])

def matpow(M, n):
    if n < 1: 
        return M
    else:
        return tf.matmul(M, matpow(M, n-1))

## Assign Devices Manually 

### All CPU Devices
Note the execution time.

In [6]:
import datetime

with tf.device("/job:local/task:0/cpu:0"):
    A = tf.random_normal(shape=[10000, 10000])
    c1 = matpow(A,n)

with tf.device("/job:local/task:1/cpu:0"):
    B = tf.random_normal(shape=[10000, 10000])
    c2 = matpow(B,n)

with tf.Session("grpc://127.0.0.1:2222") as sess:
    sum = c1 + c2
    start_time = datetime.datetime.now()
    print(sess.run(sum))
    print("Execution time: " 
          + str(datetime.datetime.now() - start_time))
          

InternalError: Dst tensor is not initialized.
	 [[Node: MatMul_1_G1 = _Recv[client_terminated=false, recv_device="/job:local/replica:0/task:0/gpu:0", send_device="/job:local/replica:0/task:0/cpu:0", send_device_incarnation=-1353772184060482797, tensor_name="edge_10_MatMul_1", tensor_type=DT_FLOAT, _device="/job:local/replica:0/task:0/gpu:0"]()]]
	 [[Node: add_G3 = _Recv[client_terminated=false, recv_device="/job:local/replica:0/task:0/cpu:0", send_device="/job:local/replica:0/task:0/gpu:0", send_device_incarnation=-5406186626203830430, tensor_name="edge_12_add", tensor_type=DT_FLOAT, _device="/job:local/replica:0/task:0/cpu:0"]()]]

### CPU and GPU
Note the execution time.

In [ ]:
with tf.device("/job:local/task:0/gpu:0"):
    A = tf.random_normal(shape=[10000, 10000])
    c1 = matpow(A,n)

with tf.device("/job:local/task:1/cpu:0"):
    B = tf.random_normal(shape=[10000, 10000])
    c2 = matpow(B,n)

with tf.Session("grpc://127.0.0.1:2222") as sess:
    sum = c1 + c2
    start_time = datetime.datetime.now()
    print(sess.run(sum))
    print("Execution time: " 
          + str(datetime.datetime.now() - start_time))

### All GPU Devices
Note the execution time.

In [ ]:
with tf.device("/job:local/task:0/gpu:0"):
    A = tf.random_normal(shape=[10000, 10000])
    c1 = matpow(A,n)

with tf.device("/job:local/task:1/gpu:0"):
    B = tf.random_normal(shape=[10000, 10000])
    c2 = matpow(B,n)

with tf.Session("grpc://127.0.0.1:2222") as sess:
    sum = c1 + c2
    start_time = datetime.datetime.now()
    print(sess.run(sum))
    print("Execution time: " 
          + str(datetime.datetime.now() - start_time))

### Auto-assign Device by TensorFlow (Round-Robin by Default)
Note the execution time.

In [ ]:
with tf.device(tf.train.replica_device_setter(worker_device="/job:worker/task:0",
                                              cluster=cluster)):
    A = tf.random_normal(shape=[10000, 10000])
    c1 = matpow(A,n)

with tf.device(tf.train.replica_device_setter(worker_device="/job:worker/task:1",
                                              cluster=cluster)):
    B = tf.random_normal(shape=[10000, 10000])
    c2 = matpow(B,n)

with tf.Session("grpc://127.0.0.1:2222") as sess:
    sum = c1 + c2
    start_time = datetime.datetime.now()
    print(sess.run(sum))
    print("Multi node computation time: " 
          + str(datetime.datetime.now() - start_time))